# Librarys

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

**Project**

A report by --- speaks to the importans of public libraries for ----. In this project we provide simple statistics and comparisons of how librabries in munusipalities relate to other (----). 
This project is in no way ment to claim a causal conection, as we are simple investigating correlations. 


Report: https://www.roskildebib.dk/sites/roskilde.ddbcms.dk/files/files/news/roskildebib_folkebibliotekets_betydning_for_borgerne_i_danmark_eng_final_0.pdf


Imports and set magics:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

import datetime

import pandas_datareader # install with `pip install pandas-datareader`
import pydst # install with `pip install git+https://github.com/elben10/pydst`


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

from pandasql import sqldf

# Read and clean data

# Importing dataset 1: Libraries by manusipalities 

**Importing data**

Viewing an importing data

In [ ]:
#Object for interacting with DST 
Dst = pydst.Dst(lang='en') # setup data loader with the langauge 'english'

In [ ]:
#Subjects within DST 
Dst.get_subjects()

In [ ]:
#List of tables within the subject Culture and leisure with index 7, and id 8
tables = Dst.get_tables(subjects=['7'])
display(tables)

**SLET DET HER UNDER**

In [ ]:
#List of variables from the tabel, BIB1 from subject index =  7, id = 8
#tables[tables.id =='BIB1']

In [ ]:
#list of variables (again?)
bib_var = Dst.get_variables(table_id = 'BIB1')
bib_var

# Variables within BNØGLE
#print(bib_var['BNØGLE'].unique())

In [ ]:
#Values of the variables  
bib_vars  = Dst.get_variables(table_id = 'BIB1')

for id in [ 'BNØGLE', 'Tid']:
    print(id)
    values = bib_vars.loc[bib_vars.id ==id, ['values']].values[0,0]
    for value in values:
        print(f'id = {value["id"]}, text = {value["text"]}')

**SLET: STOP**  \\\
Skriv anden code der viser de unike coder under BNØGLE

In [ ]:
#Eksracting the disired data
#Dictionary of variables of the disired data  (Loan. All materials)
var_pick = {'OMRÅDE': ['*'], 'BNØGLE': ['15110'], 'Tid': ['*']}

#Raw dataset 
bib_api = Dst.get_data(table_id = 'BIB1', variables = var_pick)
bib_api.sort_values(by= ['OMRÅDE', 'TID'])

bib_api.head(10)

# Importing dataset 2: number of people in the ages (OVERVEJ EN ALDERS BETINGELSE)

In [ ]:
#Subjects within DST 
Dst.get_subjects()

In [ ]:
#List of tables within the subject People        index = 0 id = 1 
tables = Dst.get_tables(subjects=['1'])
display(tables)

**SLET**

In [ ]:
#tables = [tables.id == 'BEFOLK1']
#print(tables)

#list of variables (again?)
folk_vars = Dst.get_variables(table_id = 'FOLK1A')
folk_vars

for id in ['OMRÅDE', 'Tid']:
    print(id)
    values = folk_vars.loc[folk_vars.id == id, ['values']].values[0,0]
    for value in values:
        print(f'id= {value["id"]}, text = {value["text"]}')



**SLET: STOP**

In [ ]:
#Loading the data 
var_pick = {'OMRÅDE': ['*'], 'Tid':['*']}
bef_api = Dst.get_data(table_id='FOLK1A', variables=var_pick)
bef_api.sort_values(by=['OMRÅDE'], inplace=True)

In [ ]:
bef_api.head(10)

In [ ]:
# Keeping only the 1 quarter 
I = bef_api['TID'].str.endswith('Q1')
bef_api_q1 = bef_api[I]


In [ ]:
#Renaming the time variable such that it does not include the quarter 
bef_api_q1 = bef_api_q1.rename(columns={'TID': 'tid_temp'})
bef_api_q1['tid_temp'] = bef_api_q1['tid_temp'].str.slice(stop=4)
bef_api_q1 = bef_api_q1.rename(columns={'tid_temp': 'TID'})



bef_api_q1.head(10)

# Merging the 2 datasets

In [ ]:
#The datasets

print(f'Library dataset contains data from: {np.min(bib_api.TID.unique())} to {np.max(bib_api.TID)}')
print(f'Municipalities in Library data = {len(bib_api.OMRÅDE.unique())}')

print(f'Bef dataset contains data from: {np.min(bef_api_q1.TID.unique())} to {np.max(bef_api_q1.TID)}')
print(f'Municipalities in Library data = {len(bef_api_q1.OMRÅDE.unique())}')

#Differenses in the datasets
diff_mun = [m for m in bef_api_q1.OMRÅDE.unique() if m not in bib_api.OMRÅDE.unique()]
print (f' Munisipalities in bef and not in Lib: {diff_mun}')


In [ ]:
#print(bib_api.columns)
#print(bef_api_q1.columns)

bib_api['TID'] = bib_api['TID'].astype('object')

print(bib_api.dtypes)
#print(bef_api_q1.dtypes)



In [ ]:
q = """
SELECT bef.OMRÅDE, bef.TID, bef.INDHOLD as n, bib.INDHOLD as n_bib
FROM bef_api_q1 as bef
INNER JOIN bib_api as bib
ON bib.OMRÅDE = bef.OMRÅDE AND bib.TID = bef.TID
"""

# Apply query using pandasql
bef_api_q1_transformed = sqldf(q)

# Print the transformed dataframe
print(f'The merged dataset: \n')
print(bef_api_q1_transformed)

print(f'The merged dataset, dimensions: \n')
print(f'Merged dataset contains data from: {np.min(bef_api_q1_transformed.TID.unique())} to {np.max(bef_api_q1.TID)}')
print(f'Merged dataset contains data form Municipalities = {len(bef_api_q1_transformed.OMRÅDE.unique())}')

**SLET**

In [31]:
q = """
SELECT bef.OMRÅDE as municipality, bef.TID as year, bef.INDHOLD as n, bib.INDHOLD as n_bib
FROM bef_api_q1 as bef
INNER JOIN bib_api as bib
ON bib.OMRÅDE = bef.OMRÅDE AND bib.TID = bef.TID
"""

# Apply query using pandasql
bef_api_q1_transformed = sqldf(q)


# Print the transformed dataframe
print(f'The merged dataset: \n')
print(bef_api_q1_transformed)

print(f'The merged dataset, dimensions: \n')
print(f'Merged dataset contains data from: {np.min(bef_api_q1_transformed.year.unique())} to {np.max(bef_api_q1.TID)}')
print(f'Merged dataset contains data form Municipalities = {len(bef_api_q1_transformed.municipality.unique())}')

The merged dataset: 

     municipality  year      n   n_bib
0        Aabenraa  2013  59208  345003
1        Aabenraa  2015  58904  306200
2        Aabenraa  2011  59795  403500
3        Aabenraa  2014  58970  338432
4        Aabenraa  2020  58761  180782
...           ...   ...    ...     ...
1127          Ærø  2012   6636   43328
1128          Ærø  2020   5964   21796
1129          Ærø  2013   6527   36475
1130          Ærø  2021   5960   20285
1131          Ærø  2010   6679   47241

[1132 rows x 4 columns]
The merged dataset, dimensions: 

Merged dataset contains data from: 2009 to 2023
Merged dataset contains data form Municipalities = 103


## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Working with the data** :

In [ ]:
df = bef_api_q1_transformed

In [ ]:
df['Loan per capita'] = df['n_bib'] / df['n']

In [ ]:
print(df['OMRÅDE'].unique())

In [ ]:
Mun = ['Aalborg', 'Gentofte', 'Silkeborg' ]

    # a. create the figure
fig = plt.figure()

    # b. plot
ax = fig.add_subplot(1,1,1)


for i, in Mun:
    I = df['OMRÅDE'] == i

    loan_per = df[I]['n']
    year_plot = df[I]['TID']

    ax.plot(year_plot, loan_per, label = i)



In [ ]:


# a. create the figure
fig = plt.figure()

# b. plot
ax = fig.add_subplot(1,1,1)

for i in Mun:
    I = df['OMRÅDE'] == i

    loan_per = df[I]['Loan per capita']
    year_plot = df[I]['year']

    ax.plot( loan_per, year_plot, label = i)


In [ ]:
# a. create the figure
fig = plt.figure()

# b. plot
ax = fig.add_subplot(1,1,1)

ax.plot(year_plot, loan_per)

In [ ]:
# a. create the figure
fig = plt.figure()

# b. plot
ax = fig.add_subplot(1,1,1)

ax.plot(df['n_bib'].loc['Aabenraa'], df['year'].loc['Aabenraa'])

In [ ]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.